In [46]:
import pandas as pd

with open('../data/processed/oshhamaho.txt', 'r') as f:
    kbd_sentences = f.read().split('\n')

kbd_sentences = sorted(set([sent for sent in kbd_sentences if 20 <= len(sent) <= 30]))

df = pd.DataFrame(kbd_sentences, columns=['sent'])

In [47]:
# df = df[df['sent'].str.startswith('– ')]
df

,sent
0,"( бгъэдохьэри, щэхуу)"
1,( и псалъэр зэпеудри)
2,( и псалъэр зэпеудри) 85 86
3,( и хъуреягъыр къакIухьурэ)
4,( и хьэдэм къыхуэIэбэурэ)
...,...
29906,"… ущымысхьу, си бгъэм из"
29907,…Псым уитхьэлэу уэ зэгуэрым
29908,…СыкъимыкIуэту хэт и пащхьи
29909,"…СыкъыщIокIри, мазэм соплъыр,"


In [48]:
import os
import anthropic


client = anthropic.Anthropic(
    api_key=os.getenv('ANTHROPIC_API_KEY'),
)

model_name = "claude-3-5-sonnet-20240620"

def batch_translate(bigrams):
    message = client.messages.create(
        # model="claude-3-opus-20240229",
        model=model_name,
        max_tokens=2000,
        temperature=0,
        system="Выступи в роли знатока кабардинского языка и помоги мне перевести строки на русский язык. \n",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Результат ожидаю без лишних объяснений, строго перевод на русском языке. \n\nСтроки для перевода будут разделены знаком \\n (переноса строки), в ответе перевод должен быть так же разделен \\n.  \n\nКаждая новая строка, это отдельное по смыслу предложение - соединять их и тем более пытатся их переводить вместе не стоит. \n\nНа выходе переведенных строк должно быть строго как и входных. \n\nЕсли не можешь перевести строку, ставь пустую строку \"\" чтобы остальные переведы остались под тем же индексом, это супер важно. \n\nНе пиши в ответе вводную информацию \"Перевод на русский язык:\" - только результат перевода. \n\n\nпокажу пример данных чтобы ты понимал формат. \n- на вход:\nнэхърэ нэхъ\nсыт хуэдэ\n\n- на выходе хочу получить:\nчем больше\nкакой, какая, какое\n\nследующим сообщением пришлю входные данные которые нужно будет тебе перевести. "
                    }
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": "Понял, жду строки для перевода. "
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "\n".join(bigrams)
                    }
                ]
            }
        ]
    )

    result = message.content[0].text.split('\n')
    return result

In [50]:
from tqdm import tqdm

batches = df.groupby(df.index // 100)

result_df = pd.DataFrame()

for i, (name, group) in tqdm(enumerate(batches), total=len(batches)):
    if i < 49:
        continue

    group_result = batch_translate(group['sent'].tolist())
    try:
        group_result_df = pd.DataFrame({
            'source': group['sent'].tolist(),
            'translated': group_result
        })
        group_result_df.to_csv(f'../data/processed/translated/sentences/translation_rus_{model_name}_{i}.csv', index=False)
    except Exception:
        continue

100%|██████████| 300/300 [2:48:16<00:00, 33.65s/it]   
